In [ ]:
import requests
import pandas as pd

def fetch_data(url):
    response = requests.get(url)
    data = response.json()
    if response.status_code != 200:
        print("faild to fetch data")
        return None
    return data
def parse_weather(entry):
    if entry is None:
        return None
    return {
        'Datetime': pd.to_datetime(entry['dt'], unit='s'),
        'Temperature (C)': entry['main']['temp'] - 273.15,
        'Humidity (%)': entry['main']['humidity'],
        'Wind Speed (m/s)': entry['wind']['speed'],
        'Weather': entry['weather'][0]['description']
    }
def save_to_csv(data, filename):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    print(f"{filename} saved")
def calculate_daily_stats(forecast_data):
    df_forecast = pd.DataFrame(forecast_data)
    df_forecast["Date"] = df_forecast["Datetime"].dt.date
    daily_stats =df_forecast.groupby("Date").agg({
        'Temperature (C)': ['min', 'max'],
        'Humidity (%)': ['min', 'max'],
        'Wind Speed (m/s)': ['min', 'max']
    })
    daily_stats.columns = ['Min Temperature (C)', 'Max Temperature (C)',
                           'Min Humidity (%)', 'Max Humidity (%)',
                           'Min Wind Speed (m/s)', 'Max Wind Speed (m/s)']
    daily_stats.reset_index(inplace = True)
    return daily_stats
API_KEY = "399b7d330c247da4e202f43b66680ae2"
LOCATION = "Guangzhou"
forecast_filename = "C:/Users/86185/SCNU/weather-dashboard-rendered-main (2)/weather-dashboard-rendered-main/notebooks/weather_forecast_data.csv"
current_filename = "C:/Users/86185/SCNU/weather-dashboard-rendered-main (2)/weather-dashboard-rendered-main/notebooks/weather_current_data.csv"
daily_filename = "C:/Users/86185/SCNU/weather-dashboard-rendered-main (2)/weather-dashboard-rendered-main/notebooks/daily_stats.csv"
current_url = f'http://api.openweathermap.org/data/2.5/weather?q={LOCATION}&appid={API_KEY}'
forecast_url = f'http://api.openweathermap.org/data/2.5/forecast?q={LOCATION}&appid={API_KEY}'
weather_forecast_data = fetch_data(forecast_url)
weather_current_data = fetch_data(current_url)
parse_forecast_data =[parse_weather(entry) for entry in weather_forecast_data["list"]]
parse_current_data = parse_weather(weather_current_data)
save_to_csv([parse_current_data], current_filename)
save_to_csv([parse_forecast_data], forecast_filename)
daily_stats = calculate_daily_stats(parse_forecast_data)
save_to_csv(daily_stats, daily_filename)
